# Diffusion MRI data and derivatives in the Human Connectome Project

## Diffusion MRI 

Diffusion MRI uses directionally oriented gradients and a pulsed gradient scan sequence to sensitize the MRI measurement to location- and direction-specific random diffusion of water. 

This is useful because in locations where the tissue is densely packed, diffusion of water is restricted. In locations where tissue is oriented (e.g., in axonal bundles) diffusion is less restricted along the length of the bundles than across their boundaries providing a cue for computational tractography methods.

![](images/diffusion.jpg)


In [ ]:
from pathlib import Path

# Make the path object:
cache_path = Path('/tmp/cache')

# Just because we have made a cache path object doesn't mean that the directory
# we made exists; here we check if it exists and make the directory if not.
if not cache_path.exists():
    cache_path.mkdir()


In [ ]:
from utils import ls, crawl
import nibabel as nib

In [ ]:
from cloudpathlib import S3Path, S3Client

client = S3Client(
    local_cache_dir=cache_path,
    profile_name="hcp")

hcp_base_path = S3Path(
    "s3://hcp-openaccess/",
    client=client)

In [ ]:
ls(hcp_base_path)

In [ ]:
# ls(hcp_base_path / "HCP_1200")

In [ ]:
crawl(hcp_base_path / "HCP_1200" / "100206" / "T1w" / "Diffusion")

In [ ]:
diffusion_100206 = hcp_base_path / "HCP_1200" / "100206" / "T1w" / "Diffusion" / "data.nii.gz"

In [ ]:
img_100206 = nib.load(diffusion_100206.fspath)

In [ ]:
# Careful! This requires a lot of memory! 
# img_100206.get_fdata()

## Fetching HCP diffusion data using DIPY

In addition to this direct access to the data through cloudpathlib, the HCP dMRI data can be accessed using DIPY's data fetcher system. A fetcher for HCP diffusion data has been implemented to organize each subjects data into a BIDS-compatible format. This is particularly useful if you are going to process the data using a BIDS-compatible software tool subsequently.

In [ ]:
from dipy.data.fetcher import fetch_hcp

In [ ]:
fetch_hcp(["100206"])

In [ ]:
import bids

In [ ]:
layout = bids.BIDSLayout('/home/jovyan/.dipy/HCP_1200/derivatives/hcp_pipeline/', is_derivative=True)

In [ ]:
layout.get()

## HCP tractometry derivatives

Tractometry assesses the properties of the white matter tissue along the major white matter bundles 

![](images/afq.jpg)

We have implemented a pipeline for doing this called [pyAFQ](https://yeatmanlab.github.io/pyAFQ) and we ran it on the 1,041 HCP subjects that have complete dMRI scans.

![](images/hcp_fa.jpg)


These derivatives are available through the [Open Neurodata project](https://neurodata.io/project/ocp/) (not to be confused with OpenNeuro!)

In [ ]:
from cloudpathlib import S3Path, S3Client

# Create a client that uses our cache path and that does not try to
# authenticate with S3.
client = S3Client(
    local_cache_dir=cache_path,
    no_sign_request=True)

# Now, create a cloudpath for the open neurodata's S3 bucket:
hcp_derivs_path = S3Path(
    "s3://open-neurodata/rokem/hcp1200/afq",
    client=client)

In [ ]:
crawl(hcp_derivs_path / "sub-100206")

In [ ]:
import pandas as pd

In [ ]:
tract_profiles = pd.read_csv(hcp_derivs_path / "sub-100206" / "ses-01" / "sub-100206_dwi_space-RASMM_model-CSD_desc-prob-afq_profiles.csv")

In [ ]:
tract_profiles

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(tract_profiles[tract_profiles["tractID"] == "CST_L"]["dki_fa"].values)
ax.set_xlabel("Node")
ax.set_ylabel("FA")